In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import linalg as LA
from numpy.random import default_rng
import pywt
import ham_cr
import seaborn as sns

from keras.models import Sequential
from keras import optimizers
from keras import models
from keras import layers
from keras.layers import BatchNormalization
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
# generate a complete data set of {cV, \chi_z, M_z} for one set of random Stevens parameters
# Comment: we could think to make a class that generates a full set of N training data sets (from N random Stevens parameters - then N would be given to the class). Here, the random drawing of Stevens parameters happens outside the class. 

class training_data:
    """
    Class generates and output training data: specific_heat(T), susceptibility(T) and magnetization(T, B) along specified direction(s). 
    (Optional) parameters that can be provided at instantiation: 
        point_group: Point group defined form of crystal field Hamiltonian (set to 'Oh' per default)
        N_t: Number of training sets = number of randomly chosen Stevens parameter sets. (Set to 1 per default)
        rng_seed: seed of random number generator that draws Stevens parameters (set to 1 per default)
        J, L, S: angular momentum of ion (set to J=4, L=5, S=1 per default)
        B_directions: magnetic field directions that are considered in susc and mag (set to [[0,0,1]] per default)

    Functions: 
    """

    # Bohr magneton over Boltzmann constant
    # Used to transform magnetic field B from unitsof Tesla to units of Kelvin: [muB*B/k_B] = Kelvin with [B] = Tesla
    muB_over_kB = 0.671713816 

    def __init__(self, point_group = 'Oh', N_t = 1, rng_seed = 1, J = 4, L = 5, S = 1, B_directions = [[0,0,1]]):
        self.point_group = point_group
        self.N_t = N_t
        self.rng_seed = rng_seed
        self.rg = default_rng(rng_seed)
        self.J = J
        self.L = L
        self.S = S
        self.B_directions = B_directions

    ###### define angular momentum operators Jx_op, Jy_op, Jz_op for a given J value #####
    def Jz_op(self):
        mat = np.diag(np.arange(2*self.J+1,dtype=np.float) - self.J)
        return mat

    def Jplus_op(self):
        mat = np.diag(np.sqrt((2*self.J - np.arange(2*self.J))*(np.arange(2*self.J)+1)), -1)
        return mat

    def Jminus_op(self):
        mat = np.diag(np.sqrt((2*self.J - np.arange(2*self.J))*(np.arange(2*self.J)+1)), 1)
        return mat

    def Jx_op(self):
        mat = (self.Jplus_op() + self.Jminus_op())/2.
        return mat

    def Jy_op(self):
        mat = -1j/2.*(self.Jplus_op() - self.Jminus_op())
        return mat

    def gJLS(self):
        return 1 + (self.J*(self.J + 1) + self.S*(self.S+1) - self.L*(self.L+1))/(2*self.J*(self.J + 1))

    ####### Draw random Stevens paramaters #######################################

    def generate_random_stevens(self, range):
        """
        Generated random values for Stevens parameters for given point group. 

        Parameters: 
            ppoint_group: string of point group in Schoenflies notation
            range: array of ranges for Stevens parameters [[x0_min, x0_max], [x1_min, x1_max], ...] 
        Returns: 
            stevens_params: array with random instances of Stevens parameters
        """
        # TO DO: implement error messages if range is not correct: in particular, it will get stuck if the range of the x_1, ..., x_{N-1} is not given by [-1,1]
        if self.point_group == 'Oh': # two Stevens parameters for m-3m = Oh point group
            x0 = (range[0][0] + (range[0][1] - range[0][0])*self.rg.random())*self.rg.choice([-1,1])
            x1 = range[1][0] + (range[1][1] - range[1][0])*self.rg.random()
            stevens_params = np.array([x0, x1])
        elif self.point_group == "C4v": # 5 Stevens parameters for 4mm = C4v point group
            stevens_params = np.array([1.,1.,1.,1.,1., 0.])
            while (np.sum(np.abs(stevens_params)) - np.abs(stevens_params[0]) - np.abs(stevens_params[-1]) > 1):
                stevens_params[0] = (range[0][0] + (range[0][1] - range[0][0])*self.rg.random())*self.rg.choice([-1,1])
                stevens_params[1] = range[1][0] + (range[1][1] - range[1][0])*self.rg.random()
                stevens_params[2] = range[2][0] + (range[2][1] - range[2][0])*self.rg.random()
                stevens_params[3] = range[3][0] + (range[3][1] - range[3][0])*self.rg.random()
                stevens_params[4] = range[4][0] + (range[4][1] - range[4][0])*self.rg.random()
            stevens_params[5] = 2.*self.rg.random() - 1. # only sign of x5 matters as size is determined by x1, .., x4. 
        elif self.point_group == "D3h": # 4 Stevens parameters for -6m2 = D3h point group
            stevens_params = np.array([1.,1.,1.,1., 0.])
            while (np.sum(np.abs(stevens_params)) - np.abs(stevens_params[0]) - np.abs(stevens_params[-1]) > 1):
                stevens_params[0] = (range[0][0] + (range[0][1] - range[0][0])*self.rg.random())*self.rg.choice([-1,1])
                stevens_params[1] = range[1][0] + (range[1][1] - range[1][0])*self.rg.random()
                stevens_params[2] = range[2][0] + (range[2][1] - range[2][0])*self.rg.random()
                stevens_params[3] = range[3][0] + (range[3][1] - range[3][0])*self.rg.random()
            stevens_params[4] = 2.*self.rg.random() - 1. # only sign of x5 matters as size is determined by x1, .., x4. 
        else:
            raise ValueError("This point group is not implemented.")
        return stevens_params
            
    ####### Define the crystal field Hamiltonian for given point group and J ##########
    def ham_cr(self, stevens_params):
        """
        Outputs crystal field Hamiltonian H in units of Kelvin. The units of H are set by the units of x0. We choose the range of x0 (=[1,50] Kelvin) that corresponds to [x0] = Kelvin. 

        Parameters: 
            stevens_params: array of Stevens parameters (check that length is correct). x0 has dimensions of energy (we use Kelvin) and x1, x2, ... are dimensionless in interval [-1,1].
        Returns: 
            ham_cr: crystal field Hamiltonian array 
        """
        if (self.point_group == 'Oh'):  
            if (len(stevens_params) != 2): 
                raise ValueError("Number of Stevens parameters should be 2 for point group Oh")
            if (self.J == 4):
                return ham_cr.ham_cr_PG_Oh_J_4(stevens_params[0], stevens_params[1])
            elif (self.J == 7.5):
                return ham_cr.ham_cr_PG_Oh_J_7_5(stevens_params[0], stevens_params[1])
            elif (self.J == 3.5):
                return ham_cr.ham_cr_PG_Oh_J_3_5(stevens_params[0], stevens_params[1])
            elif (self.J == 6):
                return ham_cr.ham_cr_PG_Oh_J_6(stevens_params[0], stevens_params[1])
            elif (self.J == 8):
                return ham_cr.ham_cr_PG_Oh_J_8(stevens_params[0], stevens_params[1])
            elif (self.J == 4.5):
                return ham_cr.ham_cr_PG_Oh_J_4_5(stevens_params[0], stevens_params[1])
        elif (self.point_group == 'C4v'): 
            if (len(stevens_params) != 6): 
                raise ValueError("Number of Stevens parameters should be 5+1=6 for point group C4v")
            if (self.J == 4):
                return ham_cr.ham_cr_PG_C4v_J_4(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4], stevens_params[5])
            elif (self.J == 7.5):
                return ham_cr.ham_cr_PG_C4v_J_7_5(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4], stevens_params[5])
            elif (self.J == 3.5):
                return ham_cr.ham_cr_PG_C4v_J_3_5(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4], stevens_params[5])
            elif (self.J == 6):
                return ham_cr.ham_cr_PG_C4v_J_6(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4], stevens_params[5])
            elif (self.J == 8):
                return ham_cr.ham_cr_PG_C4v_J_8(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4], stevens_params[5])
            elif (self.J == 4.5):
                return ham_cr.ham_cr_PG_C4v_J_4_5(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4], stevens_params[5])

        elif (self.point_group == 'D3h'): 
            if (len(stevens_params) != 5): 
                raise ValueError("Number of Stevens parameters should be 4+1=5 for point group D3h")
            if (self.J == 4):
                return ham_cr.ham_cr_PG_D3h_J_4(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4])
            elif (self.J == 7.5):
                return ham_cr.ham_cr_PG_D3h_J_7_5(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4])
            elif (self.J == 3.5):
                return ham_cr.ham_cr_PG_D3h_J_3_5(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4])
            elif (self.J == 6):
                return ham_cr.ham_cr_PG_D3h_J_6(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4])
            elif (self.J == 8):
                return ham_cr.ham_cr_PG_D3h_J_8(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4])
            elif (self.J == 4.5):
                return ham_cr.ham_cr_PG_D3h_J_4_5(stevens_params[0], stevens_params[1], stevens_params[2], stevens_params[3], stevens_params[4])
        else: 
            raise ValueError("This point group and/or value of J is not implemented.")   

    ####### Calculate specific heat ##################################

    def specific_heat(self, ham, T_min=2, T_max=300, T_steps=150):
        """
        Returns array of cV/kB for a single rare-earth ion over temperature range [T_min, T_max] for hamiltonian matrix ham. Note that [cV/kB] is dimensionless. To get the specific heat, multiply the result with the Boltzmann constant kB. 

        Parameters: 
            ham : hermitian Hamiltonian matrix, typically of crystal field Hamiltonian (dimension 2*J+1)
            T_min : minimal temperature in Kelvin
            T_max : maximal temprature in Kelvin
            T_steps: total number of steps in temperature range

        Returns: 
            cV_array: cV/kB for a single rare-earth ion. Array of dimension (T_steps, 2) containing (T_i, cV/kB(T_i) ),             where T_i is temperature at step i 

        """  
        T = np.linspace(T_min, T_max, T_steps) # linearly spaced temperatures
        energies = LA.eigvalsh(ham)
        energies = energies - energies[0]

        # partition function for zero field
        def Z_cr(T):
            return np.sum(np.exp(-energies/T))
        
        # specific heat expression
        def cV(T):
            return 1./(T**2) * (np.sum(energies**2 * np.exp(-energies/T))/Z_cr(T) - np.sum(energies * np.exp(-energies/T)/Z_cr(T))**2)

        cV_array = np.zeros((len(T), 2))

        #this can probably be optimized using numpy ufuncs, but it is ok for now
        for i in range(0, len(cV_array)):
            cV_array[i][0] = T[i]
            cV_array[i][1] = cV(T[i])
    
        return cV_array

    ######### Calculate magnetization ##############################

    def magnetization(self, ham_cr, B_direction, B_min=0, B_max=10, B_steps=20, T_min=2, T_max=300, T_steps=4):
        """
        Returns array of moment per R-ion mu/mu_B (over mu_B) over temperature and magnetic field range [T_min, T_max] and [B_min, B_max] for a system with zero-field Hamiltonian matrix ham_cr. Note mu/mu_B is dimensionless. The magnetic field is along B_direction (x, y, z).  
        Parameters: 
            ham_cr : hermitian Hamiltonian matrix in zero field, typically of crystal field Hamiltonian (dimension 2*J+1)
            B_direction: (B_x, B_y, B_z) triple denoting the field direction in real space
            B_min: minimal field (in Tesla)
            B_max: maximal field (in Tesla)
            B_steps: total number of steps in field range
            T_min : minimal temperature in Kelvin
            T_max : maximal temprature in Kelvin
            T_steps: total number of steps in temperature range

        Returns: 
            mag_array: induced moment on R-ion mu/muB in array of dimension (T_steps, B_steps, 2) containing (B_i, T_i, mag(B_i, T_i) ), where T_i (B_i) is temperature (field) at step i. Note that this differs by a factor of gJLS from a previous version of the function. 

        """  
        gJLS = float(self.gJLS())

        T_array = np.geomspace(T_min, T_max, T_steps)
        B_array = np.linspace(B_min, B_max, B_steps)

        B_direction = B_direction/LA.norm(B_direction)
        J_op = B_direction[0]*self.Jx_op() + B_direction[1]*self.Jy_op() + B_direction[2]*self.Jz_op()

        mag_array = np.zeros((len(B_array), len(T_array), 3))

        # this can probably be optimized using numpy ufuncs, but it is ok for now
        for B_idx in np.arange(0, len(B_array)):
            B = B_array[B_idx]
            ham = ham_cr - gJLS*self.muB_over_kB*J_op*B
            energies, eigenstates = LA.eigh(ham)
            energies = energies - energies[0]
            
            for T_idx in range(0, len(T_array)):
                T = T_array[T_idx]
                ZB = np.sum(np.exp(-energies/T))
                # mag = \mu/\mu_B is moment per R-ion over Bohr magneton. mag is dimensionless. 
                mag = gJLS/ZB*np.sum([np.dot(np.conjugate(eigenstates[:,i]), np.dot(J_op, eigenstates)[:, i])*np.exp(-energies[i]/T) for i in range(0, len(energies))])
    
                mag_array[B_idx][T_idx][0] = B_array[B_idx]
                mag_array[B_idx][T_idx][1] = T_array[T_idx]
                mag_array[B_idx][T_idx][2] = mag

        return mag_array
    
    ########### Calculate magnetic susceptibility ############################
    
    def susceptibility(self, ham_cr, B_direction, B=0.0001, T_min=1, T_max=300, T_steps=300):
        """
        Calculated and returns magnetic susceptibility chi_a = mu/(mu_B*B) (units of 1/T) over temperature range [T_min, T_max] for zero-field Hamiltonian matrix ham_cr. Here, mu is the induced moment on the R-ion, mu_B is the Bohr magneton and B the magnetic field. The direction is a=B_direction .
        Parameters: 
            ham_cr : hermitian Hamiltonian matrix in zero field, typically a crystal field Hamiltonian (dimension 2*J+1)
            B_direction: (B_x, B_y, B_z) triple denoting the field direction in real space
            B: B field used in susceptibility calculation (in Tesla). Should be << all other energy scales. 
            T_min : minimal temperature in Kelvin
            T_max : maximal temprature in Kelvin
            T_steps: total number of steps in temperature range
            
        Returns: 
            susc_array: array of dimension (T_steps, 2) containing (T_i, mu(T_i)/(muB*B), where T_i is temperature at step i and mu(T_i)/mu_B = mag(T_i) is the field induced moment on the R-ion. Note that this differs by a factor of gJLS from a previous version of the function. 

        """  
        gJLS = float(self.gJLS())

        T_array = np.linspace(T_min, T_max, T_steps)

        B_direction = B_direction/LA.norm(B_direction)
        J_op = B_direction[0]*self.Jx_op() + B_direction[1]*self.Jy_op() + B_direction[2]*self.Jz_op()

        susc_array = np.zeros((len(T_array), 2))

        # this can probably be optimized using numpy ufuncs, but it is ok for now  
        # B is given in units of T, ham is in units of K.
        ham = ham_cr - gJLS*self.muB_over_kB*J_op*B
        energies, eigenstates = LA.eigh(ham)
        energies = energies - energies[0]
            
        for T_idx in range(0, len(T_array)):
                T = T_array[T_idx]
                ZB = np.sum(np.exp(-energies/T))
                # mag = mu/mu_B, where \mu is the field induced moment on the R-ion
                mag = gJLS/ZB*np.sum([np.dot(np.conjugate(eigenstates[:,i]), np.dot(J_op, eigenstates)[:, i])*np.exp(-energies[i]/T) for i in range(0, len(energies))])

                susc_array[T_idx][0] = T_array[T_idx]
                # susc = mag/B = \mu/(\mu_B B) has units of 1/T
                susc_array[T_idx][1] = mag/B 

        return susc_array

    ######## Output training data into files #################

    def output_all_data(self, coeff, cV_T_range = [1, 300, 100], susc_T_range = [1, 300, 100], mag_T_range = [1, 300, 4], mag_B_range = [0.5, 10, 20]):
        """
        Write training data to file
        Parameters: 
            Stevens_range: array of ranges for Stevens parameters [[x0_min, x0_max], [x1_min, x1_max], ...] 
        Optional parameters: 
            cV_T_range: [T_min, T_max, T_steps] array for specific heat calculation
            susc_T_range: [T_min, T_max, T_steps] array for susceptibility calculation
            mag_T_range: [T_min, T_max, T_steps] array for magnetization calculation
            mag_B_range: [B_min, B_max, B_steps], where B_steps is the number of B points within range [B_min, B_max]
            coeff: Stevens coefficients to generate data
        Returns: 
            stevens_params_all: array with parameter values of Stevens parameters
            cV_data_all: array with specific heat values
            susc_data_all: array with susceptibility values
            mag_data_all: array with magnetization values

        """
        stevens_params_all = [] 
        cV_data_all = []
        susc_data_all = []
        mag_data_all = []
        
        stevens_params = coeff
        stevens_params_all.append(stevens_params) # use a list to store all Stevens parameters. Since different point groups have different number of Stevens parameters, the tuples that are stored have different length. 
        ham_cr = self.ham_cr(stevens_params) # crystal field Hamiltonian for given random Stevens parameters

        # generate specific heat data and store in cV_data
        cV_data_all.append(self.specific_heat(ham_cr, T_min = cV_T_range[0], T_max = cV_T_range[1], T_steps = cV_T_range[2]))

        B_direction_steps = len(self.B_directions)

        # generate susceptibility data and store in susc_data (for all B_directions)
        susc_data = np.zeros((susc_T_range[2], 1 + B_direction_steps))
        mag_data = np.zeros((mag_B_range[2], mag_T_range[2], 2 + B_direction_steps))

        for B_direction_idx in range (0, B_direction_steps):
            B_direction = self.B_directions[B_direction_idx]
            susc_array = self.susceptibility(ham_cr, B_direction, B = 0.0001, T_min = susc_T_range[0], T_max = susc_T_range[1], T_steps = susc_T_range[2])
            mag_array = self.magnetization(ham_cr, B_direction, B_min = mag_B_range[0], B_max = mag_B_range[1], B_steps = mag_B_range[2], T_min = mag_T_range[0], T_max = mag_T_range[1], T_steps = mag_T_range[2])

            for T_idx in range (0, len(susc_array)):
                if (B_direction_idx == 0): 
                    susc_data[T_idx][0] = susc_array[T_idx][0]
                susc_data[T_idx][1 + B_direction_idx] = susc_array[T_idx][1]

            for B_idx in range (0, mag_B_range[2]):
                for T_idx in range(0, mag_T_range[2]):
                    if (B_direction_idx == 0):
                        mag_data[B_idx][T_idx][0] = mag_array[B_idx][T_idx][0]
                        mag_data[B_idx][T_idx][1] = mag_array[B_idx][T_idx][1]
                    mag_data[B_idx][T_idx][2 + B_direction_idx] = mag_array[B_idx][T_idx][2]
        susc_data_all.append(susc_data)
        mag_data_all.append(mag_data)
        return stevens_params_all, cV_data_all, susc_data_all, mag_data_all
            

In [3]:
def cwt(data, channels=6, wavelet='morl'):
    """
    Applies the continuous wavelet transformation on each dataset (susc, spec heat, magnetization at 4 temps) individually
    and combines each convolution into a multi-channel image to be fed to the CNN
    Args:
        data: 1D array containing each dataset
        channels: number of channels in the output image (susc, spec heat, magnetization at 4 temps)
        waveletname: mother wavelet function to be used Can be any from 
            https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html, specifically pywt.wavelist()

    Returns:
        Input data reshaped into a (shape x shape) image with 'channels' channels
    """
    shape = data.shape[1]//channels
    size = data.shape[0]
    scales = np.linspace(1, 200, shape)

    data_cwt = np.ndarray(shape=(size, shape, shape, channels), dtype=np.float16)
    for i in range(size):
        if (i % 1000 == 0):
            print('.', end='')

        # generating each of the channels
        for j in range(channels):
            signal = data[i][j*shape: shape+(j*shape)]
            coeff, freq = pywt.cwt(signal, scales, wavelet, 1)
            data_cwt[i, :, :, j] = coeff

    return data_cwt

In [4]:
num_training_examples = 1
J = 15/2 #4
L = 6 #5
S = 3/2 #1
group = 'C4v'
B_directions = [[1,0,0],[0,0,1]] #,[0,0,1]]
Stevens_range = [[0.5, 50],[-1,1],[-1,1],[-1,1],[-1,1]]

td = training_data(group, num_training_examples, 123, J, L, S, B_directions)

#out[0] # Stevens parameters
#out[1] # specific heat [[T_i, cV^(0)_i], [T_i, cV^(1)_i], ..., [T_i, cV^(N_t-1)_i] ], i = 1, ..., T_steps
#out[2] # susceptibility [[T_i, susc^(0)_{0,i}, susc^{(0)_{1,i}, ..., susc^(0)_{B_direction-1,i}}], ...], i = 1, ..., T_steps
#out[3] # magnetization [[[B_j, T_i, M^(0),{0,i}, M^(0)_{1,i,j}, ..., M^(0)_{B_direction-1,i,j}], ... ]], j = 1, .., B_steps; i = 1, ..., T_steps

In [5]:
dir_pos = './TrainingData_C4v_7.5_1/'
dir_neg = './TrainingData_C4v_7.5_-1/'

data_pos = dir_pos + 'generated_data_{}.csv'
targets_pos = dir_pos + 'generated_targets_{}.csv'
data_neg = dir_neg + 'generated_data_{}.csv'
targets_neg = dir_neg + 'generated_targets_{}.csv'

In [6]:
x_train = np.array(pd.read_csv(data_pos.format(1000), header=None))
y_train = np.array(pd.read_csv(targets_pos.format(1000), header=None))
channels = len(x_train[0])//64

In [7]:
x_mean_pos = np.load(dir_pos + 'x_mean_{}_{}.npy'.format(group, J))
y_mean_pos = np.load(dir_pos + 'y_mean_{}_{}.npy'.format(group, J))
y_std_pos = np.load(dir_pos + 'y_std_{}_{}.npy'.format(group, J))

# x_mean_neg = np.load(dir_neg + 'x_mean_{}_{}.npy'.format(group, J))
# y_mean_neg = np.load(dir_neg + 'y_mean_{}_{}.npy'.format(group, J))
# y_std_neg = np.load(dir_neg + 'y_std_{}_{}.npy'.format(group, J))

In [8]:
from keras.models import load_model
model_pos = load_model(dir_pos + '{}_{}_1_model.h5'.format(group, J))
# model_neg = load_model(dir_neg + '{}_{}_-1_model.h5'.format(group, J))

In [9]:
def predict(data):
    """
    """
    data_cwt = cwt(np.array([data]), channels=channels)
    data_cwt -= x_mean_neg
    pred_neg = model_neg.predict(data_cwt)
    pred_neg = (pred_neg * y_std_neg) + y_mean_neg
    
    # data_cwt = cwt(np.array([data]), channels=channels)
    # data_cwt -= x_mean_pos
    # pred_pos = model_pos.predict(data_cwt)
    # pred_pos = (pred_pos * y_std_pos) + y_mean_pos

    return pred_pos

In [10]:
def mse_generated_data(x_real, pred_y):
    """
    """
    B_temps = 3
    # real_out = td.output_all_data(real_y, cV_T_range = [1, 300, 64], susc_T_range = [1, 300, 64], mag_T_range = [1, 300, B_temps], mag_B_range = [0, 10, 64])
    pred_out = td.output_all_data(pred_y, cV_T_range = [1, 300, 64], susc_T_range = [1, 300, 64], mag_T_range = [1, 300, B_temps], mag_B_range = [0, 10, 64])
    
    error = 0
    error += sum([(yr - yp)**2 for (yr, yp) in zip(np.array(x_real)[0*64: 1*64], np.array(pred_out[1])[:,:,1].flatten())])
    for i in range(len(B_directions)): # size of B_directions
        error += sum([(yr -yp)**2 for (yr, yp) in zip(np.array(x_real)[(1+i)*64: (i+2)*64], np.array(pred_out[2])[:,:,i+1].flatten())])
    for i in range(B_temps): # T step for magnetization
        for j in range(len(B_directions)): # size of B_directions
            index = (i*2)+j
            error += sum([(yr -yp)**2 for (yr, yp) in zip(np.array(x_real)[(index+3)*64: (index+4)*64], np.array(pred_out[3])[:,:,i,j+2].flatten())])
    return error / len(x_train[0])

In [11]:
N = 4000
y_real_pos = np.array(pd.read_csv(targets_pos.format(N), header=None))
x_pos_flat = np.array(pd.read_csv(data_pos.format(N), header=None))
# y_real_neg = np.array(pd.read_csv(targets_neg.format(N), header=None))
# x_neg = np.array(pd.read_csv(data_neg.format(N), header=None))

x_pos = cwt(x_pos_flat, channels=channels)
x_pos -= x_mean_pos
y_pred_pos = model_pos.predict(x_pos)
y_pred_pos = (y_pred_pos * y_std_pos) + y_mean_pos
# x_neg = cwt(x_neg, channels=channels)
# x_neg -= x_mean_neg
# y_pred_neg = model_neg.predict(x_neg)
# y_pred_neg = (y_pred_neg * y_std_neg) + y_mean_neg

....

In [12]:
errors_pos = [mse_generated_data(x_pos_flat[i], y_pred_pos[i]) for i in range(len(y_real_pos))]
# errors_neg = [mse_generated_data(y_real_neg[i], y_pred_neg[i]) for i in range(len(y_real_neg))]

In [13]:
np.mean(errors_pos)

0.2554236520556009

In [13]:
res = pd.DataFrame()

for i in range(len(y_pred_pos[0])):
    res['pred{}'.format(i)] = np.concatenate([y_pred_pos[:,i]]) #, y_pred_neg[:,i]])
    res['real{}'.format(i)] = np.concatenate([y_real_pos[:,i]]) #, y_real_neg[:,i]])
res['error'] = np.concatenate([errors_pos]) #, errors_neg])

In [14]:
from sklearn import metrics
for i in range(len(y_pred_pos[0])-1):
    print('Mean absolute error: {}'.format(metrics.mean_absolute_error(res['real{}'.format(i)], res['pred{}'.format(i)])))
    print('Mean squared error: {}'.format(metrics.mean_squared_error(res['real{}'.format(i)], res['pred{}'.format(i)])))
    print('Explained varience score: {}'.format(metrics.explained_variance_score(res['real{}'.format(i)], res['pred{}'.format(i)])))
    print('r^2 score: {}'.format(metrics.r2_score(res['real{}'.format(i)], res['pred{}'.format(i)])))
    print()

Mean absolute error: 1.380133178249534
Mean squared error: 3.6891363295913475
Explained varience score: 0.9856941233725036
r^2 score: 0.9822699106672403

Mean absolute error: 0.021735490317100026
Mean squared error: 0.0008538631129992888
Explained varience score: 0.9904862336229929
r^2 score: 0.9873608518352461

Mean absolute error: 0.038225996280905124
Mean squared error: 0.002946243037411409
Explained varience score: 0.955681373376696
r^2 score: 0.9547375154781441

Mean absolute error: 0.031306389075998434
Mean squared error: 0.0018373407601389893
Explained varience score: 0.972091102731856
r^2 score: 0.9720050649127008

Mean absolute error: 0.05887875581226877
Mean squared error: 0.009289278842763794
Explained varience score: 0.871069044324308
r^2 score: 0.8620612345786324



In [15]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

last_real = [0 if i < 0 else 1 for i in res['real{}'.format(5)]]
last_pred = [0 if i < 0 else 1 for i in res['pred{}'.format(5)]]
report = classification_report(last_real, last_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.93      0.93      1932
           1       0.93      0.94      0.93      2068

    accuracy                           0.93      4000
   macro avg       0.93      0.93      0.93      4000
weighted avg       0.93      0.93      0.93      4000



In [1]:
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
from matplotlib import ticker, cm
from matplotlib.ticker import MultipleLocator

res_sample = res.sample(2000)

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Palatino'] + plt.rcParams['font.serif']
plt.rc('axes', linewidth=1.2)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

fig, ax = plt.subplots(ncols=6, nrows=1, figsize=(13,2.5), gridspec_kw={"width_ratios":[1,1,1,1,1,0.05]})
plt.tight_layout(pad=0.4)
ax2 = plt.axes([0.897, 0.2, 0.046, 0.24])
ax3 = plt.axes([0.13, 0.2, 0.046, 0.24])


for i in range(len(y_pred_pos[0])-1):
    # ax[i].xaxis.set_tick_params(which='major', direction='in', top='on')
    # ax[i].yaxis.set_tick_params(which='major', direction='in', right='on')

    p2 = ax[i].scatter(res_sample['real{}'.format(i)], res_sample['pred{}'.format(i)], marker='o', alpha=0.8, s=72./fig.dpi, c=res_sample['error'], norm=LogNorm(), cmap=cm.magma)

    ax[i].set_xlabel('True $x_{}$'.format(i), fontsize=10)
    

    if (i == 0):
        ax[i].xaxis.set_minor_locator(MultipleLocator(5))
        ax[i].yaxis.set_minor_locator(MultipleLocator(5))
        ax[i].set_ylim([-3,53])
        ax[i].set_xlim([-3,53])
        ax[i].set_ylabel('Predicted $\hat{x}_i$', fontsize=10)
    else:
        ax[i].xaxis.set_major_locator(MultipleLocator(0.5))
        ax[i].xaxis.set_minor_locator(MultipleLocator(0.25))
        ax[i].yaxis.set_major_locator(MultipleLocator(0.5))
        ax[i].yaxis.set_minor_locator(MultipleLocator(0.25))
        ax[i].set_ylim([-1.1,1.1])
        ax[i].set_xlim([-1.1,1.1])
    
    lims = [
        np.min([ax[i].get_xlim(), ax[i].get_ylim()]),  # min of both axes
        np.max([ax[i].get_xlim(), ax[i].get_ylim()]),  # max of both axes
    ]
    ax[i].plot(lims, lims, 'k--', alpha=0.5, linewidth=0.5)
    ax[i].set_xlim(lims)
    ax[i].set_ylim(lims)
        
ax2.hist([x for x in res_sample['error'] if x < 1], color="black", density=False, alpha=1) # 1000 examples
ax2.set_xlabel('MSE', fontsize=8)
ax2.set_ylabel('Count', fontsize=8)
ax2.xaxis.set_label_position('top') 
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)

res_sample = res.sample(400)
ax3.scatter(res_sample['real{}'.format(5)], res_sample['pred{}'.format(5)], marker='o', c='k', alpha=1, s=18./fig.dpi)
ax3.set_xlabel('True $x_5$', fontsize=8)
ax3.set_ylabel('Pred. $\hat{x}_5$', fontsize=8)
ax3.xaxis.set_label_position('top') 
ax3.tick_params(axis='x', labelsize=6)
ax3.tick_params(axis='y', labelsize=6)

cax = ax[5]
ip = InsetPosition(ax[4], [1.05,0,0.05,1]) 
cax.set_axes_locator(ip)
cb1 = fig.colorbar(p2, cax=cax, ax=[ax[0],ax[1],ax[2],ax[3],ax[4]])
cb1.set_alpha(1)
cb1.draw_all()
cax.set_ylabel('MSE', fontsize=8)


plt.savefig('C4v_7.5_results_combined.png', dpi=300, transparent=False, bbox_inches='tight')
plt.show()

NameError: name 'res' is not defined